<a href="https://colab.research.google.com/github/radosty/radosty.github.io/blob/main/extractingeuropeanunemploymentandinflationdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
import pandas as pd

# List of ISO3 codes for European countries
european_countries = [
    "ALB", "AND", "AUT", "BLR", "BEL", "BIH", "BGR", "HRV", "CYP", "CZE", "DNK",
    "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "ISL", "IRL", "ITA", "XKX", "LVA",
    "LIE", "LTU", "LUX", "MLT", "MDA", "MCO", "MNE", "NLD", "MKD", "NOR", "POL",
    "PRT", "ROU", "SMR", "SRB", "SVK", "SVN", "ESP", "SWE", "CHE", "UKR", "GBR"
]

# Indicators for unemployment and inflation
indicators = {
    "SL.UEM.TOTL.ZS": "Unemployment rate",
    "FP.CPI.TOTL.ZG": "Inflation rate"
}

data = []

for indicator_code, indicator_name in indicators.items():
    # Construct the API URL
    url = f"https://api.worldbank.org/v2/country/{';'.join(european_countries)}/indicator/{indicator_code}?date=2010:2023&format=json&per_page=10000"

    # Fetch data from the World Bank API
    response = requests.get(url)
    if response.status_code == 200:
        api_data = response.json()
        if api_data and len(api_data) > 1:
            records = api_data[1]
            for record in records:
                if record['value'] is not None:
                    year = int(record['date'])
                    if 2010 <= year <= 2023:
                        data.append({
                            'Country': record['country']['value'],
                            'Year': year,
                            'Indicator': indicator_name,
                            'Value': record['value']
                        })
    else:
        print(f"Failed to fetch {indicator_name} data:", response.status_code)

# Convert to DataFrame
df = pd.DataFrame(data)

# Pivot the data to have Unemployment and Inflation as separate columns
df_wide = df.pivot(index=['Country', 'Year'], columns='Indicator', values='Value').reset_index()

# Sort data for better readability
df_wide.sort_values(by=['Country', 'Year'], inplace=True)

print("Data Retrieved:")
display(df_wide)

# Save the data to a CSV file
df_wide.to_csv("european_unemployment_inflation_2010_2023.csv", index=False)
print("Data saved to 'european_unemployment_inflation_2010_2023.csv'")



Data Retrieved:


Indicator,Country,Year,Inflation rate,Unemployment rate
0,Albania,2010,3.626047,14.086
1,Albania,2011,3.429123,13.481
2,Albania,2012,2.031593,13.376
3,Albania,2013,1.937621,15.866
4,Albania,2014,1.625865,18.055
...,...,...,...,...
563,United Kingdom,2019,1.738105,3.737
564,United Kingdom,2020,0.989487,4.472
565,United Kingdom,2021,2.518371,4.826
566,United Kingdom,2022,7.922049,3.730


Data saved to 'european_unemployment_inflation_2010_2023.csv'
